# Customer Segmentation

Bir e-ticaret şirketi müşterilerini segmentlere ayırıp bu segmentlere göre pazarlama stratejileri belirlemek istiyor.

Buna yönelik olarak müşterilerin davranışlarını tanımlayacağız ve bu davranışlarda öbeklenmelere göre gruplar oluşturacağız.

Yani ortak davranışlar sergileyenleri aynı gruplara alacağız ve bu gruplara özel satış ve pazarlama teknikleri geliştirmeye çalışacağız.

**Veri Seti Hikayesi**

https://archive.ics.uci.edu/ml/datasets/Online+Retail+II

Online Retail II isimli veri seti İngiltere merkezli online bir satış mağazasının 01/12/2009 - 09/12/2011 tarihleri arasındaki satışlarını içeriyor.

Bu şirket hediyelik eşya satıyor. Promosyon ürünleri gibi düşünebilir.

Müşterilerinin çoğu da toptancı.

**Değişkenler**

- InvoiceNo: Fatura numarası. Her işleme yani faturaya ait eşsiz numara. Eğer bu kod C ile başlıyorsa işlemin iptal edildiğini ifade eder.
- StockCode: Ürün kodu. Her bir ürün için eşsiz numara.
- Description: Ürün ismi
- Quantity: Ürün adedi. Faturalardaki ürünlerden kaçar tane satıldığını ifade etmektedir.
- InvoiceDate: Fatura tarihi ve zamanı. 
- UnitPrice: Ürün fiyatı (Sterlin cinsinden)
- CustomerID: Eşsiz müşteri numarası
- Country: Ülke ismi. Müşterinin yaşadığı ülke.



# Data Understanding 

In [1]:
#Gerekli olan kütüphanelerin import işlemleri yapılmıştır.
import pandas as pd
import numpy as np
import seaborn as sns

#Tüm sutünları ve satırları gözlemlemek için kullandığımız kod.
pd.set_option('display.max_columns', None); pd.set_option('display.max_rows', None);

#virgulden sonra gösterilecek olan sayı sayısı
pd.set_option('display.float_format', lambda x: '%.0f' % x)
import matplotlib.pyplot as plt

In [2]:
#Veri seti okuma işlemi gerçekleştirildi.
df_2010_2011 = pd.read_excel("online_retail_II.xlsx", sheet_name = "Year 2010-2011")

In [3]:
#Veri seti boyutu büyük olduğu için okuma işlemi uzun sürdü bu sebeple veri setini kopyalama işlemi yaptık.
df = df_2010_2011.copy()

# Pandas kütüphanesi ile veriyi anlamaya çalışınız.

In [4]:
#Veri setine ait ilk 5 gözlem birimine eriştik.
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,3,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,3,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3,17850,United Kingdom


In [5]:
#essiz ürün sayısı nedir?
df["Description"].nunique()

4223

In [6]:
#hangi üründen kaçar tane var?
df["Description"].value_counts().head()

WHITE HANGING HEART T-LIGHT HOLDER    2369
REGENCY CAKESTAND 3 TIER              2200
JUMBO BAG RED RETROSPOT               2159
PARTY BUNTING                         1727
LUNCH BAG RED RETROSPOT               1638
Name: Description, dtype: int64

In [7]:
#en çok sipariş edilen ürün hangisi?
df.groupby("Description").agg({"Quantity":"sum"}).head()

,Quantity
Description,
20713,-400
4 PURPLE FLOCK DINNER CANDLES,144
50'S CHRISTMAS GIFT BAG LARGE,1913
DOLLY GIRL BEAKER,2448
I LOVE LONDON MINI BACKPACK,389


In [8]:
#yukarıdaki çıktıyı nasıl sıralarız?
df.groupby("Description").agg({"Quantity":"sum"}).sort_values("Quantity", ascending = False).head()

,Quantity
Description,
WORLD WAR 2 GLIDERS ASSTD DESIGNS,53847
JUMBO BAG RED RETROSPOT,47363
ASSORTED COLOUR BIRD ORNAMENT,36381
POPCORN HOLDER,36334
PACK OF 72 RETROSPOT CAKE CASES,36039


In [9]:
#toplam kaç fatura kesilmiştir?
df["Invoice"].nunique()

25900

In [10]:
#fatura basina toplam kac para kazanilmistir? (iki değişkeni çarparak yeni bir değişken oluşturmak gerekmektedir)
#kaç para kazanıldığını bulmak için "adet(Quantity) * fiyat(Price)" yapılması gerekir. 
df["TotalPrice"] = df["Quantity"]*df["Price"]

In [11]:
#Dataframe "Total Price" değişkeni eklenmiştir. Bu değişken ilk 5 gözlem üzerinden incelenmiştir. 
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,3,17850,United Kingdom,15
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3,17850,United Kingdom,20
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,3,17850,United Kingdom,22
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3,17850,United Kingdom,20
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3,17850,United Kingdom,20


In [12]:
#Fatura numarasına (Invoice) göre gruplayıp "Total Price" değişkenin toplamını getirildi. 
#Bu şekilde fatura başına toplam kaç para kazanıldı hesaplanmıştır.
df.groupby("Invoice").agg({"TotalPrice":"sum"}).head()

,TotalPrice
Invoice,
536365,139
536366,22
536367,279
536368,70
536369,18


In [13]:
#en pahalı ürünler hangileri?
#Ürünlerin ismine göre groupby yapılarak "Price" değişkenin max değeri alınmıştır ve azalan şekilde sıralanmıştır.
df.groupby("Description").agg({"Price":"max"}).sort_values("Price", ascending = False).head()

,Price
Description,
Manual,38970
AMAZON FEE,17836
Adjust bad debt,11062
POSTAGE,8143
DOTCOM POSTAGE,4505


In [14]:
#en pahalı ürünler hangileri? 
#Farklı bir çözüm olarak direk dataframe üzerinden "Price" değişkenine göre azalan şekilde sıralama yapılmaktadır. 
df.sort_values("Price", ascending = False).head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
222681,C556445,M,Manual,-1,2011-06-10 15:31:00,38970,15098,United Kingdom,-38970
524602,C580605,AMAZONFEE,AMAZON FEE,-1,2011-12-05 11:36:00,17836,nan,United Kingdom,-17836
43702,C540117,AMAZONFEE,AMAZON FEE,-1,2011-01-05 09:55:00,16888,nan,United Kingdom,-16888
43703,C540118,AMAZONFEE,AMAZON FEE,-1,2011-01-05 09:57:00,16454,nan,United Kingdom,-16454
16356,C537651,AMAZONFEE,AMAZON FEE,-1,2010-12-07 15:49:00,13541,nan,United Kingdom,-13541


In [15]:
#hangi ülkeden kac sipariş geldi?
#Ülke kategorik bir değişken olduğu için kategorik değişkenlerin sınıflarını value_counts() fonksiyonu ile saydırabiliriz.
df["Country"].value_counts().head()

United Kingdom    495478
Germany             9495
France              8558
EIRE                8196
Spain               2533
Name: Country, dtype: int64

In [16]:
#hangi ülke ne kadar kazandırdı?
#Ülkelere göre groupby yapılarak "Total Price" değişkenini toplamı alınarak hangi ülke ne kadar bırakmış öğrenilmiştir.
df.groupby("Country").agg({"TotalPrice":"sum"}).sort_values("TotalPrice", ascending = False).head()

,TotalPrice
Country,
United Kingdom,8187806
Netherlands,284662
EIRE,263277
Germany,221698
France,197422


In [17]:
#en çok iade alan ürün hangisidir?
#Invoice değişkeninde yer alan kodlar başında "C" ifadesi iade anlamına gelmektedir.
#Veri setinde şu anda na değerler olduğu için str.contains komutu içinde na değerlerini False olarak tanımlayıp görmezden gel diyoruz.
#Adet işlemi için de "Quantity" değişkenini azalan şekilde sıralarsak yakaladığımız Invoice değişkenine göre en çok iadeleri buluruz.
returned = df[df["Invoice"].str.contains("C",na=False)]
returned.sort_values("Quantity", ascending = True).head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
540422,C581484,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,2011-12-09 09:27:00,2,16446,United Kingdom,-168470
61624,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74215,2011-01-18 10:17:00,1,12346,United Kingdom,-77184
4268,C536757,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,-9360,2010-12-02 14:23:00,0,15838,United Kingdom,-281
160145,C550456,21108,FAIRY CAKE FLANNEL ASSORTED COLOUR,-3114,2011-04-18 13:08:00,2,15749,United Kingdom,-6539
160144,C550456,21175,GIN + TONIC DIET METAL SIGN,-2000,2011-04-18 13:08:00,2,15749,United Kingdom,-3700


# Data Preparation

# Eksik Gözlem Analizi

In [18]:
# Hiç eksik gözlem var mı sorusunu sormaktadır.
df.isnull().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
TotalPrice          0
dtype: int64

In [19]:
# Elimdeki veri setinde yaklaşık olarak 500000 gözlemden oluşan bir veri seti var bu nedenle eksik gözlemleri silebilirim.
df.dropna(inplace = True)

In [20]:
#Silme işleminden sonra tekrardan kontrol ediyorum eksik gözlem var mı?
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
TotalPrice     0
dtype: int64

In [21]:
#Boyut bilgisine erişilmektedir.
df.shape

(406830, 9)

In [22]:
#Çeyrekliklerini(Kartiller) kendimiz belirleyerek betimsel istatistiklere bakılmaktadır. Veri seti hakkında incelemeler yapılmaktadır.
df.describe([0.05,0.01,0.25,0.50,0.75,0.80,0.90,0.95,0.99]).T

,count,mean,std,min,1%,5%,25%,50%,75%,80%,90%,95%,99%,max
Quantity,406830,12,249,-80995,-2,1,2,5,12,12,24,36,120,80995
Price,406830,3,69,0,0,0,1,2,4,4,7,8,15,38970
Customer ID,406830,15288,1714,12346,12415,12626,13953,15152,16791,17095,17719,17905,18212,18287
TotalPrice,406830,20,428,-168470,-10,1,4,11,20,21,35,67,200,168470


# Aykırı Gözlem Analizi

In [23]:
# Aykırı gözlem analizinde en yaygın kullanılan method bir alt limit ve üst limit belirleyerek baskılamaktır.
# Burada 1. ve 3. çeyreklikler göz önüne alınarak IQR hesaplanır ve alt, üst sınırlar burada belirlenir. 
# Neden sadece bu değişkenler çünkü yukarıdaki kodda Customer ID hariç aykırı gözlem analizi olduğu saptanmıştır.
#Burada aykırı gözlemlere ucundan değinilerek bu problem ortadan kaldırılmıştır.
for feature in ["Quantity","Price","TotalPrice"]:

    Q1 = df[feature].quantile(0.01)
    Q3 = df[feature].quantile(0.99)
    IQR = Q3-Q1
    upper = Q3 + 1.5*IQR
    lower = Q1 - 1.5*IQR

    if df[(df[feature] > upper) | (df[feature] < lower)].any(axis=None):
        print(feature,"yes")
        print(df[(df[feature] > upper) | (df[feature] < lower)].shape[0])
    else:
        print(feature, "no")

Quantity yes
948
Price yes
846
TotalPrice yes
1030


# RFM Skorları ile Müşteri Segmentasyonu

Recency, Frequency, Monetary ifadelerinin baş harflerinden oluşur.

Müşterilerin satın alma alışkanlıkları üzerinden pazarlama ve satış stratejileri belirlemeye yardımcı olan bir tekniktir.

- Recency (yenilik): Müşterinin son satın almasından bugüne kadar geçen süre

    -- Diğer bir ifadesiyle “Müşterinin son temasından bugüne kadar geçen süre” dir.

    -- Bugünün tarihi - Son satın alma

    -- Örnek verecek olursak bugün bu analizi yapıyorsak bugünün tarihi - son 	ürün satın alma tarihi.

    -- Bu örneğin 20 olabilir 100 olabilir. Biliriz ki 20 olan müşteri daha sıcaktır. Daha son zamanlarda bizimle teması olmuştur.

- Frequency (Sıklık): Toplam satın alma sayısı.

- Monetary (Parasal Değer): Müşterinin yaptığı toplam harcama.

In [24]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,3,17850,United Kingdom,15
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3,17850,United Kingdom,20
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,3,17850,United Kingdom,22
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3,17850,United Kingdom,20
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3,17850,United Kingdom,20


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406830 entries, 0 to 541909
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      406830 non-null  object        
 1   StockCode    406830 non-null  object        
 2   Description  406830 non-null  object        
 3   Quantity     406830 non-null  int64         
 4   InvoiceDate  406830 non-null  datetime64[ns]
 5   Price        406830 non-null  float64       
 6   Customer ID  406830 non-null  float64       
 7   Country      406830 non-null  object        
 8   TotalPrice   406830 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 31.0+ MB


In [27]:
df.shape

(406830, 9)

# Recency Score Hesaplama

**Recency Score = Bugünün tarihi - Son satın alma**

Bugün nedir? Şimdi eğer günümüz tarihini alırsak bu durumda arada çok ciddi bir fark olacaktır.

Bu sebeple örnek olması açısından kendimize bu veri setinin yapısına göre bir "bugün" belirleyelim.

Bu günü veri setinin maksimum günü olarak belirleyebiliriz.

Son kayıt düşülen güne göre segmentasyon yapabiliriz.

Yapılacak olan işlem analizin yapılacağı günden, Customer ID değişkenine göre groupby yapmak ve InvoiceDate değişkeninin maksimum değerleri alınarak bu iki değeri birbirinden çıkararak Recency Score hesaplanmış olacaktır. 

In [58]:
#Alışverişin yapıldığı ilk gün
df["InvoiceDate"].min()

Timestamp('2010-12-01 08:26:00')

In [59]:
#Alışverişin yapıldığı son gün
df["InvoiceDate"].max()

Timestamp('2011-12-09 12:50:00')

In [60]:
#Analiz için gerekli import işlemi yapılarak max değer analizi yaptığımız gün olarak belirlenmiştir. Bu değer today_date olarak atanmıştır.
import datetime as dt
today_date = dt.datetime(2011,12,9)
today_date

datetime.datetime(2011, 12, 9, 0, 0)

In [72]:
#Customer ID değerine göre groupby işlemi yapılıp Invoice Date değişkenin max değerleri getirildi. 
#Yani son alışveriş yapılan tarihler geldi. Müşteri ID sine göre son yapılan alışveriş tarihlerine ulaştık. 
df.groupby("Customer ID").agg({"InvoiceDate":"max"})

,InvoiceDate
Customer ID,
12346,2011-01-18 10:17:00
12347,2011-12-07 15:52:00
12348,2011-09-25 13:13:00
12349,2011-11-21 09:51:00
12350,2011-02-02 16:01:00
12352,2011-11-03 14:37:00
12353,2011-05-19 17:47:00
12354,2011-04-21 13:11:00
12355,2011-05-09 13:49:00


In [73]:
# Customer ID değişkeni stringe dönüştürülmüştür.
df["Customer ID"] = df["Customer ID"].astype(int)

In [75]:
# Analizin yapıldığı gün - Son alışveriş yapılan tarih işlemi yapıldığında recency değerini yakaladık.
temp_df = (today_date - df.groupby("Customer ID").agg({"InvoiceDate":"max"}))
temp_df

,InvoiceDate
Customer ID,
12346,324 days 13:43:00
12347,1 days 08:08:00
12348,74 days 10:47:00
12349,17 days 14:09:00
12350,309 days 07:59:00
12352,35 days 09:23:00
12353,203 days 06:13:00
12354,231 days 10:49:00
12355,213 days 10:11:00


In [77]:
# Analiz yapıldı ve burada InvoıceDate değişkeninin adını Recency olarak değiştirilmektedir.
temp_df.rename(columns={"InvoiceDate": "Recency"}, inplace = True)
temp_df

,Recency
Customer ID,
12346,324 days 13:43:00
12347,1 days 08:08:00
12348,74 days 10:47:00
12349,17 days 14:09:00
12350,309 days 07:59:00
12352,35 days 09:23:00
12353,203 days 06:13:00
12354,231 days 10:49:00
12355,213 days 10:11:00


In [78]:
#Yukarıda yapılan işlem sonucunda Recency değişkeninde saat değerlerini silmek istiyorum.
#Bunu yapabilmek için, daha önce burası Invoice Date değişkeni ve veri tipi datetime olduğu için days'leri seçip saatleri silebiliriz. 
recency_df = temp_df["Recency"].apply(lambda x: x.days)
recency_df

Customer ID
12346    324
12347      1
12348     74
12349     17
12350    309
12352     35
12353    203
12354    231
12355    213
12356     21
12357     32
12358      0
12359      6
12360     51
12361    286
12362      2
12363    108
12364      6
12365    290
12367      3
12370     50
12371     43
12372     70
12373    310
12374     24
12375      1
12377    314
12378    128
12379     80
12380     20
12381      3
12383    183
12384     27
12386    336
12388     14
12390     78
12391     20
12393     71
12394     62
12395     14
12397     34
12398     44
12399    118
12401    302
12402    322
12403     48
12405    147
12406     21
12407     48
12408     31
12409     77
12410    300
12412     73
12413     65
12414    216
12415     23
12417      2
12418    111
12420     62
12421     14
12422     94
12423     -1
12424    161
12425     77
12426    193
12427     10
12428     24
12429      8
12430     42
12431     34
12432     41
12433     -1
12434     83
12435     78
12436     98
12437      0


# Frequency Score Hesaplama
**Frequency (Sıklık): Toplam satın alma sayısı**

Yapılacak olan işlem Customer ID değişkenine göre groupby yapmak ve Invoice değişkeninin toplamı getirilerek Frequency Score hesaplanmış olacaktır. 

In [46]:
#Customer ID ye göre ve Invoice'a gruopby yap ve Invoice değişkenini say.
temp_df = df.groupby(["Customer ID","Invoice"]).agg({"Invoice":"count"})
temp_df.head()

Invoice
Customer ID Invoice         
12346       541431         1
            C541433        1
12347       537626        31
            542237        29
            549222        24

In [47]:
#Customer ID ye göre gruopby yap ve Invoice değişkenini topla. Yani her müşteri kaç alışveriş yapmış toplamda bunu bulmuş oluyoruz.
temp_df.groupby("Customer ID").agg({"Invoice":"sum"}).head()

,Invoice
Customer ID,
12346,2
12347,182
12348,31
12349,73
12350,17


In [48]:
#Burada yukarıda yapmış olduğumuz işlemi bir değişkene atayıp "Invoice" yazan değişkeni de "Frequency" olarak değiştirip gözlemliyoruz.
freq_df = temp_df.groupby("Customer ID").agg({"Invoice":"sum"})
freq_df.rename(columns={"Invoice": "Frequency"}, inplace = True)
freq_df.head()

,Frequency
Customer ID,
12346,2
12347,182
12348,31
12349,73
12350,17


# Monetary Score Hesaplama
**Monetary (Parasal Değer): Müşterinin yaptığı toplam harcama.**

Yapılacak olan işlem Müşteri denildiği için Customer ID değişkenine göre groupby işlemi yapılıp TotalPrice değişkeninin toplamı getirilerek Monetary Score hesaplanmış olacaktır.

In [52]:
monetary_df = df.groupby("Customer ID").agg({"TotalPrice":"sum"})
monetary_df.head()

,TotalPrice
Customer ID,
12346,0
12347,4310
12348,1797
12349,1758
12350,334


In [56]:
#Sutün adlandırması değiştirildi.
monetary_df.rename(columns = {"TotalPrice": "Monetary"}, inplace = True)
monetary_df.head()

,Monetary
Customer ID,
12346,0
12347,4310
12348,1797
12349,1758
12350,334


RFM Analizi için gerekli olan parametreler hesaplanmıştır. Şimdi bu parametereler üzerinden bazı işlemler ve yorumlar yapıyor olacağız. 

In [79]:
# Her bir parametrenin boyutlarına bakıldı. 
print(recency_df.shape,freq_df.shape,monetary_df.shape)

(4372,) (4372, 1) (4372, 1)


In [83]:
#Parametreleri bir dataframe olarak concat fonksiyonu yardımı ile birleştirdik.
rfm = pd.concat([recency_df, freq_df, monetary_df],  axis=1)
rfm.head()

,Recency,Frequency,Monetary
Customer ID,,,
12346,324,2,0
12347,1,182,4310
12348,74,31,1797
12349,17,73,1758
12350,309,17,334


RFM Analizinde yaygın olarak kullanılmakta olan bir tablo bulunmaktadır. Aşağıda bunu sizinle paylaşmaktayım. Genelde bu tablolarda 2 parametre kullanılmaktadır. Bunlar Recency ve Frequency'dir. Burada bu parametrelere değerler verilmektedir. Değerler görüldüğü üzere 1 ile 5 arasında olan değerlerdir. 5 çok iyi, 1 ise çok kötü anlamına gelmektedir. Bu tabloda görüldüğü üzere müşteriler segmentlere ayrılmış durumdadır. Parametre değerlerine bakılarak müşterilerin segmentleri belirlenmektedir. Bizlerin burada dikkat etmesi gereken grup aslında herkesin düşündüğü şampiyonlar grubu değil bu gruplar zaten sürekli olarak bizi ziyaret ediyor ve alışveriş yapıp para bırakıyor. Burada en dikkat gerektiren sınıf "can't loose them" sınıfıdır. Çünkü bu kişiler bizi nerdeyse hiç ziyaret etmiyor ve çok fazla alışveriş yapıyor yani frequency değerleri 5 gördüğünüz üzere bu nedenle bu sınıfa odaklanmamız gerekiyor. Çünkü bu grupta yer alan müşteriler bizi bıraktı bırakacak bu nedenle dikkat gerektirmekte ve bu gruptaki müşterileri gerekli analizler yaparak bize kazandırmalıyız. Aynı zamanda bir de "need attention" grubu bulunmakta Recency değeri 3, Frequency değeri 3 olan gruba ve uyku halinde olan gruba özel mailler atarak, promosyonlar yaparak canlandırıp tabloda sağa veya yukarıya doğru çıkarmamız gerekmektedir. Kısacası bu tabloyu bu şekilde açıklayabiliriz. RFM Analizinin temel mantığının anlatıldığı tablodur.  


<img src = 'rfm-segments.png' />


Aşağıdaki yer alan kod parçasında elde ettiğimiz parametreleri skorlara ayırmamız gerekmektedir. Az önce yukarıda tabloyu anlatırken değindiğim değerlerden bahsediyorum. Burada dikkatinizi çeken nokta Recency değeri olacaktır. Çünkü tersten bir skorlama yapılmaktadır. Bunun sebebi ise örneğin siz analiz yapılmadan 1 gün önce bizleri ziyaret ederek alışveriş yaptınız, diğer müşterimiz ise 100 gün önce bizleri ziyaret ederek alışveriş yaptığını düşünelim. Bu durumda Recency değeri **Analiz yapılan gün - Son Alışveriş yapılan gün** formülü ile hesaplandığında burada siz bizi 1 gün önce bizi ziyaret edip alışveriş yaptığınız hesaplanacaktır. Yani yakın zamanda alışveriş yaptığınız için **recency puanınız: 5** olacaktır. Diğer müşterimiz ise analiz yapılmadan 100 gün önce alışveriş yaptığı için **recency puanı 1 olacaktır.** Bu nedenle recency değeri düşük olana 5 puan recency değeri yüksek olana 1 puan verilmektedir. Diğer iki parametre zaten satış arttıkça ve ödenen toplam para arttıkça skor değeri de artacaktır. 

In [84]:
# Buradaki skorlama işlemi aslında pandas içinde .cut fonk. kullanılarak bölünmüştür.
rfm["RecencyScore"] = pd.qcut(rfm['Recency'], 5, labels = [5, 4, 3, 2, 1])
rfm["FrequencyScore"] = pd.qcut(rfm['Frequency'], 5, labels = [1,2,3,4,5])
rfm["MonetaryScore"] = pd.qcut(rfm['Monetary'], 5, labels = [1,2,3,4,5])

In [86]:
#Skorlama işlemi sonrası dataframe ilk 5 gözlem biriminin incelenmesi
rfm.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore
Customer ID,,,,,,
12346,324,2,0,1,1,1
12347,1,182,4310,5,5,5
12348,74,31,1797,2,3,4
12349,17,73,1758,4,4,4
12350,309,17,334,1,2,2


In [87]:
#Skorları yanyana yazdırmak için yapılmaktadır.
(rfm['RecencyScore'].astype(str) + 
 rfm['FrequencyScore'].astype(str) + 
 rfm['MonetaryScore'].astype(str)).head()

Customer ID
12346    111
12347    555
12348    234
12349    444
12350    122
dtype: object

In [88]:
#Dataframe e RFM_SCORE sutünü eklenmiştir.
rfm["RFM_SCORE"] = rfm['RecencyScore'].astype(str) + rfm['FrequencyScore'].astype(str) + rfm['MonetaryScore'].astype(str)
rfm.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
Customer ID,,,,,,,
12346,324,2,0,1,1,1,111
12347,1,182,4310,5,5,5,555
12348,74,31,1797,2,3,4,234
12349,17,73,1758,4,4,4,444
12350,309,17,334,1,2,2,122


In [89]:
#Betimsel istatistikleri incelenerek bazı yorumlar yapılmaktadır.
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
Recency,4372,91,101,-1,15,49,142,372
Frequency,4372,93,232,1,17,42,102,7983
Monetary,4372,1898,8219,-4288,293,648,1612,279489


In [90]:
#En iyi müşteriler gösterilmektedir.
rfm[rfm["RFM_SCORE"] == "555"].head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
Customer ID,,,,,,,
12347,1,182,4310,5,5,5,555
12359,6,254,6246,5,5,5,555
12362,2,274,5155,5,5,5,555
12417,2,198,3579,5,5,5,555
12433,-1,420,13376,5,5,5,555


In [91]:
#En kötü müşteriler gösterilmektedir.
rfm[rfm["RFM_SCORE"] == "111"].head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
Customer ID,,,,,,,
12346,324,2,0,1,1,1,111
12353,203,4,89,1,1,1,111
12361,286,10,190,1,1,1,111
12401,302,5,84,1,1,1,111
12402,322,11,226,1,1,1,111


In [92]:
#Burada rfm skorlarına göre sınıflar atanmaktadır. Bu sınıfların hangi skor aralığında olacağı aşağıda belirtilmiştir. 
#Örnek olarak Hibernating sınıfı şu skor değerlerine sahiptir. 
#Recency değeri 1-2, Frequency değeri 1-2 olanlardır diğer sınıflarda bu şekilde okunmaktadır.
#Buraya sadece Recency ve Frequency eklenmesinin sebebi tabloda sadece bu iki parametre yer aldığı için ancak Monetary de yanlarına eklenebilir.
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At Risk',
    r'[1-2]5': 'Can\'t Loose',
    r'3[1-2]': 'About to Sleep',
    r'33': 'Need Attention',
    r'[3-4][4-5]': 'Loyal Customers',
    r'41': 'Promising',
    r'51': 'New Customers',
    r'[4-5][2-3]': 'Potential Loyalists',
    r'5[4-5]': 'Champions'
}

In [93]:
#Yukarıda tanımlanan seg_map'i dataframe dahil etme işlemi yapılmıştır.
rfm['Segment'] = rfm['RecencyScore'].astype(str) + rfm['FrequencyScore'].astype(str)
rfm['Segment'] = rfm['Segment'].replace(seg_map, regex=True)
rfm.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
Customer ID,,,,,,,,
12346,324,2,0,1,1,1,111,Hibernating
12347,1,182,4310,5,5,5,555,Champions
12348,74,31,1797,2,3,4,234,At Risk
12349,17,73,1758,4,4,4,444,Loyal Customers
12350,309,17,334,1,2,2,122,Hibernating


In [94]:
#Segmentlere göre groupby yaparak elde edilen (kodda belirtilen) 3 parametrenin ortalama ve kaç adet olduğunu getirmektedir. 
rfm[["Segment", "Recency","Frequency","Monetary"]].groupby("Segment").agg(["mean","count"]).head()

Recency       Frequency       Monetary      
                  mean count      mean count     mean count
Segment                                                    
About to Sleep      51   328        16   328      412   328
At Risk            164   577        57   577      950   577
Can't Loose        141    81       184    81     2346    81
Champions            4   620       287   620     6788   620
Hibernating        211  1066        13  1066      369  1066

In [95]:
#Bu segmentteki grup dikkat gerektiren grublardan birisiydi. Bunları yakalama işlemi yapılmıştır.
rfm[rfm["Segment"] == "Need Attention"].head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
Customer ID,,,,,,,,
12372,70,52,1298,3,3,4,334,Need Attention
12413,65,40,694,3,3,3,333,Need Attention
12446,56,58,1002,3,3,4,334,Need Attention
12454,52,30,-0,3,3,1,331,Need Attention
12458,70,38,947,3,3,4,334,Need Attention


In [96]:
#Yeni müşterilere ait Customer ID (indeks) değerlerine erişilmiştir.
#Bu değerler müşteri departmanı ile paylaşılıp yeni müşterilere özel promosyonlar ve mailler bu ID'ler sayesinde atılabilir.
rfm[rfm["Segment"] == "New Customers"].index

Int64Index([12367, 12442, 12587, 12660, 12798, 12882, 12966, 13017, 13068,
            13079, 13147, 13188, 13255, 13298, 13404, 13436, 14219, 14287,
            14354, 14385, 14480, 14520, 14569, 14601, 14777, 14785, 14804,
            14836, 14853, 14860, 14865, 15101, 15195, 15206, 15619, 15773,
            15783, 15793, 15992, 16000, 16446, 16500, 16569, 16597, 16620,
            16789, 16852, 16988, 16989, 17044, 17436, 17468, 17737, 17929,
            17942, 18030, 18058, 18174, 18273, 18282],
           dtype='int64', name='Customer ID')

In [97]:
#Dikkat gerektiren müşterilerin Customer ID (indeks) değerlerine erişilmiştir.
#Bu değerler müşteri departmanı ile paylaşılıp dikkat gerektiren müşterilere özel promosyonlar ve mailler bu ID'ler sayesinde atılabilir.
rfm[rfm["Segment"] == "Need Attention"].index

Int64Index([12372, 12413, 12446, 12454, 12458, 12475, 12512, 12571, 12631,
            12638,
            ...
            18104, 18126, 18136, 18150, 18160, 18170, 18171, 18192, 18205,
            18228],
           dtype='int64', name='Customer ID', length=208)

In [98]:
#Yeni bir dataframe oluşturularak içerisine Need Attention grubuna ait müşterilerin ID bilgileri atılmıştır.
new_df = pd.DataFrame()
new_df["NewCustomerID"] = rfm[rfm["Segment"] == "Need Attention"].index

In [99]:
#İlk 5 gözlem gözlemlenmiştir.
new_df.head()

,NewCustomerID
0,12372
1,12413
2,12446
3,12454
4,12458


In [101]:
#Atanan bu ID'ler excel çıktısı alınarak müşteri departmanı ile paylaşılmaya hazır hale getirilmiştir.
new_df.to_excel("new_customers.xlsx")

# Yorumlar ve Alınacak olan Aksiyonlar

In [104]:
rfm[["Segment", "Recency","Frequency","Monetary"]].groupby("Segment").agg(["mean","count","max"]).head(20)

Recency            Frequency             Monetary        \
                       mean count  max      mean count   max     mean count   
Segment                                                                       
About to Sleep           51   328   70        16   328    29      412   328   
At Risk                 164   577  372        57   577   124      950   577   
Can't Loose             141    81  337       184    81   548     2346    81   
Champions                 4   620   10       287   620  7983     6788   620   
Hibernating             211  1066  372        13  1066    29      369  1066   
Loyal Customers          31   825   70       162   825  1212     2581   825   
Need Attention           50   208   70        42   208    58      833   208   
New Customers             5    60   10         8    60    14      660    60   
Potential Loyalists      14   502   30        35   502    58      877   502   
Promising                21   105   30         8   105    14      413   105   

                            
                       max  
Segment                     
About to Sleep        7331  
At Risk              11057  
Can't Loose          10217  
Champions           279489  
Hibernating          21536  
Loyal Customers     123725  
Need Attention       11582  
New Customers        12394  
Potential Loyalists  26763  
Promising             6749

Yorumlarımı yukarıda yer alan betimsel istatistiklere göre gerçekleştireceğim. Seçilecek olan 4 segment şunlardır;

- **Champions**
- **About to Sleep**
- **Need Attention**
- **Can't Loose**

**Champions**
- bu segmentte 620 kişi bulunmakta,
- ortalama olarak en son alışverişleri 4 gün önce gerçekleşmiş,
- alışveriş sıklıkları 287, toplamda 7983 alışverişleri var,
- 279489 dolar/TL harcamaları olmuş.

**About to Sleep**
- bu segmentte 328 kişi bulunmakta,
- ortalama olarak en son alışverişleri 51 gün önce gerçekleşmiş,
- alışveriş sıklıkları 16, toplamda 29 alışverişleri var,
- 7331 dolar/TL harcamaları olmuş.

**Need Attention**
- bu segmentte 208 kişi bulunmakta,
- ortalama olarak en son alışverişleri 50 gün önce gerçekleşmiş,
- alışveriş sıklıkları 42, toplamda 58 alışverişleri var,
- 11582 dolar/TL harcamaları olmuş.

**Can't Loose**
- bu segmentte 81 kişi bulunmakta,
- ortalama olarak en son alışverişleri 141 gün önce gerçekleşmiş,
- alışveriş sıklıkları 184, toplamda 548 alışverişleri var,
- 10217 dolar/TL harcamaları olmuş.

**AKSİYON**

**Champions** segmentindeki kişiler ile iletişim kurmak kendilerini değerli ve takdir edilmiş kişiler olarak hissettirecektir. Bu müşteriler muhtemelen toplam gelirlerin orantısız olarak yüksek bir yüzdesini oluşturmaktadır ve bu nedenle onları mutlu etmeye odaklanmak en büyük öncelik olmalıdır. Bireysel tercihlerini ve yakınlıklarını daha fazla analiz ederek, daha kişiselleştirilmiş mesajlaşma için ek fırsatlar sağlayacaktır. Örneğin doğum günlerini kutlama gibi kişisel mesajlar gönderilebilir. Yeni ürünler için erken benimsiyor olabilirler onlara yeni ürünlerimizi ücretsiz olarak ileterek markamızı tanıtmamıza yardımcı olabilirler.

**About to sleep** segmentindeki kişilere değerli kaynaklarımızı paylaşarak, popüler ürünleri / yenilemeleri indirimli olarak önerip, onlarla yeniden bağlantı kurup müşterimiz yapabiliriz.

**Need Attention** segmentindeki kişilere sınırlı süreli teklifler yapabiliriz bu onları bizimle alışveriş yapmaya doğru itecektir, geçmiş satın alımlara dayanarak tavsiye ürünlerde bulunabiliriz. Bu şekilde onları yeniden etkinleştirerek alışveriş yapmalarını sağlayabiliriz.

**Can't Loose** önem verilmesi gerekilen segmentlerden birisidir. Bu segment bizim kaybetmek istemediğimiz segmenttir. Son alışverişleri 141 gün önce yapılmış olması kötü bir sonuçtur. Ancak toplam yapılan alışveriş sayısı oldukça yüksek bir rakamdır 548 adet toplam satış olmuş. Aslında alışveriş yapıyorlar ama tek seferde veya birkaç seferde girip toplu bir şekilde alışveriş yapıp uzun bir süre tekrar alışveriş yapmıyorlar. Bu segmentte yer alan kişilere yenilemeler, daha yeni ürünleri onlara önererek veya bu sınıfa özel promosyonlar, küçük çaplı para puanlar verilerek geri kazanabiliriz, burada yer alan kişileri rekabete sokmamalıyız, onlarla konuşarak mailler ve smsler atarak kazanabiliriz. Neyin yanlış gittiğini öğrenmek için anketler yaparak anket sonuçlarından düzeltmeler yapılabiliriz.